In [4]:
# 분별 날짜 파일에 있는 날짜 기준으로 DB에서 데이터 받아와서 저장하기

import MySQLdb
import os
import numpy as np
import csv

# MySQL DB 연결
db = MySQLdb.connect('210.102.142.13',"root", "witlab8*", "cas_db")
c = db.cursor()

# 분별 날짜
# date = np.genfromtxt('../data/date.csv', delimiter=',', dtype='str')
date = np.genfromtxt('../data/test_date.csv', delimiter=',', dtype='str')

# csv 파일로 내보내기
# w = open('../data/db_connect_data.csv', 'w', encoding='utf-8')
w = open('../data/db_connect_test_data.csv', 'w', encoding='utf-8')
wr = csv.writer(w)

# 각 날짜별 데이터들 DB에서 가져오고 csv 파일로 저장
# swr, mwr, lwr data 받아옴
for j in range(len(date)):
    sql = "select time(date),cas_swr, cas_mwr, cas_lwr from natural_tracker left outer join cas_wave_ratio using(date) where date(date) = '"+ str(date[j]) + "' order by time(date)"
    c.execute(sql)
    rows = c.fetchall()
    
    # 오전 6:00:00 데이터부터 읽어들이기. 그 순간의 index를 start에 저장
    start = 0
    
    for i in range(len(rows)):
        if(str(rows[i][0]) == '6:00:00'):
            start = i
            print(rows[start][0])
            break;
        if(str(rows[i][0]) == '6:01:00'):
            start = i
            print(rows[start][0])
            break;

    # start에 저장된 index부터 772개 데이터 가져와서 저장
    for l in range(start, start+772):
        wr.writerow([rows[l][1], rows[l][2], rows[l][3]])
    print(date[j] + " saved.")
        
w.close()
db.close()

6:00:00
2018-04-19 saved.
6:00:00
2018-04-21 saved.
6:00:00
2018-04-26 saved.
6:00:00
2018-04-27 saved.
6:00:00
2018-04-28 saved.
6:00:00
2018-04-29 saved.
6:00:00
2018-04-30 saved.
6:00:00
2018-05-04 saved.
6:00:00
2018-05-05 saved.
6:00:00
2018-05-08 saved.
6:00:00
2018-05-10 saved.
6:00:00
2018-05-14 saved.
6:00:00
2018-05-19 saved.
6:00:00
2018-05-21 saved.
6:00:00
2018-05-24 saved.
6:00:00
2018-05-25 saved.
6:00:00
2018-05-26 saved.
6:00:00
2018-05-27 saved.
6:00:00
2018-05-28 saved.
6:00:00
2018-05-29 saved.
6:00:00
2018-05-30 saved.
6:00:00
2018-05-31 saved.
6:00:00
2018-06-01 saved.
6:00:00
2018-06-02 saved.
6:00:00
2018-06-03 saved.
6:00:00
2018-06-05 saved.
6:00:00
2018-06-06 saved.
6:00:00
2018-06-07 saved.
6:00:00
2018-06-08 saved.
6:00:00
2018-06-09 saved.
6:00:00
2018-06-11 saved.
6:00:00
2018-06-13 saved.
6:00:00
2018-06-16 saved.
6:00:00
2018-06-17 saved.
6:00:00
2018-06-18 saved.
6:00:00
2018-06-24 saved.
6:00:00
2018-06-25 saved.


In [7]:
# import_data = np.loadtxt('../data/db_connect_data.csv', delimiter=',')
import_data = np.loadtxt('../data/db_connect_test_data.csv', delimiter=',')

one = import_data[:,0] # swr
two = import_data[:,1] # mwr
thr = import_data[:,2] # lwr

swr = []
mwr = []
lwr = []

for cnt in range(len(date)): # 전체 데이터 날짜 수 (행)
    temp = []
    temp2 = []
    temp3 = []
    for i in range(772): # 한 날짜의 데이터들 한 행에 저장 (열)
        temp.append(one[i+772*cnt])
        temp2.append(two[i+772*cnt])
        temp3.append(thr[i+772*cnt])
    swr.append(temp)
    mwr.append(temp2)
    lwr.append(temp3)

# w = open('../data/swr_ratio_calculation.csv', 'w', encoding='utf-8')
w = open('../data/swr_ratio_test_calculation.csv', 'w', encoding='utf-8')
wr = csv.writer(w)

for i in range(len(date)):
    all_hap = 0
    swr_hap = 0
    ratio = 0
    grade = 0
    
    for j in range(len(swr[i])):
        swr_hap += swr[i][j]
        all_hap += swr[i][j] + mwr[i][j] + lwr[i][j]
    
    ratio = round(swr_hap / all_hap * 100, 2)
    
    if(ratio >= 19.00):
        grade += 1
    if(ratio >= 20.00):
        grade += 1
    if(ratio >= 22.00):
        grade += 1
    if(ratio >= 23.00):
        grade += 1
    
    wr.writerow([ratio, grade])
w.close()

In [8]:
import_data = np.genfromtxt('../data/swr_ratio_calculation.csv', delimiter=',', dtype='float')
import_data2 = np.genfromtxt('../data/swr_ratio_test_calculation.csv', delimiter=',', dtype='float')
w = open('../data/swr_ratio_classification_input.csv', 'w', encoding='utf-8')
wr = csv.writer(w)

for i in range(len(import_data)):
    wr.writerow(import_data[i][:])
for i in range(len(import_data2)):
    wr.writerow(import_data2[i][:])
w.close()

In [1]:
# 텐서플로우 모델 생성 위한 import

import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
tf.set_random_seed(777)

# 초기값 선정 xavier 알고리즘
def xavier_init(n_inputs, n_outputs, uniform=True):
    
    if uniform:
        init_range = tf.sqrt(6.0 / (n_inputs + n_outputs))
        return tf.random_uniform_initializer(-init_range, init_range)
    else:
        stddev = tf.sqrt(3.0 / (n_inputs + n_outputs))
        return tf.truncated_normal_initializer(stddev=stddev)

# 단파장 비율 학습 및 테스트 (편차 분포)

import_data = np.genfromtxt('../data/swr_ratio_classification_input.csv', delimiter=',', dtype='float')

x_data = []
for i in range(len(import_data)):
    temp = []
    temp.append(import_data[i][0])
    x_data.append(temp)
    
# scaler = MinMaxScaler(feature_range=(0, 1))
# x_data = scaler.fit_transform(x_data)
test_x = x_data[85:]
x_data = x_data[:85]

y_data = []
test_y = []

for i in range(85):
    temp = []
    temp.append(import_data[i][1])
    y_data.append(temp)

raw_y = y_data
    
for i in range(85, len(import_data)):
    temp = []
    temp.append(import_data[i][1])
    test_y.append(temp)


y_data = ohe.fit_transform(y_data)
y_data = y_data.toarray();

X = tf.placeholder("float", [None, 1])
Y = tf.placeholder("float", [None, 5])

nb_classes = 5

# W = tf.Variable(tf.random_normal([1, nb_classes]), name='weight')
# b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# Xavier Initializer 추가 코드
W = tf.get_variable("W", shape=[1, nb_classes], initializer=xavier_init(1, nb_classes))
b = tf.Variable(tf.zeros([nb_classes]))

H = tf.nn.softmax(tf.matmul(X, W) + b)

cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(H), axis=1))

optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

with tf.Session() as sess:
    accuracy = 0
    sess.run(tf.global_variables_initializer())

    for step in range(19001):
        sess.run(optimizer, feed_dict={X:x_data, Y:y_data})
        if step % 1000 == 0:
            print(step, sess.run(cost, feed_dict={X:x_data, Y:y_data}))

    print('-----------------------------')

    print('train_data = ', len(x_data), 'test_data = ', len(test_x))

    for i in range(len(test_x)):
        a = sess.run(H, feed_dict={X:[test_x[i]]})
        print(sess.run(tf.argmax(a, 1)), test_y[i])
#         print(a[0][0], ', ', a[0][1], ', ', a[0][2], ', ', a[0][3], ', ', a[0][4])
        if(sess.run(tf.argmax(a, 1)) ==test_y[i]):
            accuracy += 1
    print("accuracy = ", float(accuracy / len(test_x) * 100))

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected

0 nan
1000 nan
2000 nan
3000 nan
4000 nan
5000 nan
6000 nan
7000 nan
8000 nan
9000 nan
10000 nan


KeyboardInterrupt: 